the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 114 ms


In [3]:
#importing in all clinical_variable files
date= '22112018'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/%s_longdf_preImp.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 12 s


In [4]:
big_df.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
0,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,28448
1,200012.0,2153-12-23,Neg/Not_tested,nitrite,pos/neg category,0 days 00:00:00.000000000,nit,28448
2,200012.0,2153-12-23,None,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,28448
3,200012.0,2153-12-23,1,daily_sofa,daily_sofa_score,0 days 00:00:00.000000000,sofa,28448
4,200012.0,2153-12-23,0.0,dobutamine,y/n,0 days 00:00:00.000000000,dobutamine,28448


time: 18.1 ms


In [5]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


,count,unique,top,freq
label,,,,
leukocyte,14668,2,Neg/Not_tested,12449
norepinephrine,35398,2,1.0,23637
vasopressin,18146,2,0.0,14240
gender,14668,2,M,8033
phenylephrine,28124,2,1.0,15776
rrt,14668,2,0.0,13448
nitrite,14668,2,Neg/Not_tested,14247
dobutamine,16003,2,0.0,14430
dopamine,19501,2,0.0,13803


time: 1.93 s


In [6]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
#df["B"] = df["A"].astype('category')

None      8387
Oxygen    3986
Mech      2295
Name: value, dtype: int64

time: 516 ms


In [7]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

time: 1.48 s


In [8]:
#big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 1.47 s


In [9]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].describe()

count     14668
unique        3
top        None
freq       8387
Name: value, dtype: object

time: 470 ms


In [10]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',
           'first_admit_age']

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2/fio2']

both_value=['calcium',
'sodium',
'wbc']

time: 6.48 ms


In [11]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

time: 3.48 ms


In [12]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 4.46 ms


In [13]:
len(dict1.keys())

37

time: 3.95 ms


### to do: need to convert all dtypes to a numeric type taht pereserves nan. 

In [14]:
categorical=['leukocyte','nitrite','vent_recieved', 'ethnicity','gender']

big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:]
big_noCat= big_df.loc[big_df.loc[:,'label'].isin(dict1.keys()),:]
big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 


time: 2min 46s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [15]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 1.19 s


In [16]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values='value', columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 1.22 s


In [17]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values='value', columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 343 ms


In [18]:
print(len(table),len(table2),len(table3))

14668 14668 14618
time: 1.12 ms


In [19]:
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')

time: 37.6 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [20]:
print(len(list(worst_df)), #40- icustay_id - label = 38. with 

big_df['label'].nunique(),
     
set(big_df['label'].unique())- set(worst_df)   )


41 42 {'leukocyte', 'sodium', 'gender', 'ethnicity', 'wbc', 'calcium', 'vent_recieved', 'nitrite'}
time: 625 ms


In [21]:
big_max= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]

time: 672 ms


In [22]:
big_categorical.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
0,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,28448
1,200012.0,2153-12-23,Neg/Not_tested,nitrite,pos/neg category,0 days 00:00:00.000000000,nit,28448
2,200012.0,2153-12-23,None,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,28448
7,200012.0,2153-12-23,asian,ethnicity,NaN,0 days 00:00:00.000000000,pt_info,28448
9,200012.0,2153-12-23,F,gender,NaN,0 days 00:00:00.000000000,pt_info,28448


time: 11 ms


In [23]:
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label') #need to convert to long format. should be one row per patient per time. 

time: 51.3 ms


In [24]:
worst_df=pd.merge(worst_df, big_categorical, left_on='icustay_id', right_on='icustay_id',how='left')

time: 30.7 ms


In [25]:
worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
worst_df['nitrite']=worst_df['nitrite'].astype("category")
worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")
worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
worst_df['gender']=worst_df['gender'].astype("category")

time: 15.2 ms


In [26]:
list(worst_df)

['icustay_id',
 'bands',
 'bilirubin',
 'bun',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'first_admit_age',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'norepinephrine',
 'pco2',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'temperature',
 'vasopressin',
 'weight',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'pao2/fio2',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender',
 'leukocyte',
 'nitrite',
 'vent_recieved']

time: 2.72 ms


In [27]:
baselinePts= final_pt_df2.loc[final_pt_df2.loc[:,'final_bin'].isin(["C_pos/A_full","C_neg/A_partial"]),"icustay_id"]
worst_df2= worst_df.loc[worst_df.loc[:,'icustay_id'].isin(baselinePts),:]

time: 14 ms


In [28]:
date= '22112018'
pd.DataFrame(worst_df).to_csv(Path(
    wd+'/data/processed/%s_worstdf_preImp.csv' %(date)),index=False)

pd.DataFrame(worst_df2).to_csv(Path(
    wd+'/data/processed/%s_worstdf2_preImp.csv' %(date)),index=False)

time: 859 ms


In [29]:
len(worst_df2)

7778

time: 2.63 ms


In [30]:
# big_categorical["icustay_id"].isin(icustay_id)

time: 680 µs


In [31]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

time: 635 µs


In [32]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

time: 879 µs


In [33]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


time: 1.15 ms


In [34]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 713 µs


In [35]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

time: 801 µs


In [36]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2

time: 673 µs
